In [13]:
import requests
from bs4 import BeautifulSoup
import datetime
import os
import pymysql

In [15]:
BASE_URL = 'https://www.fmkorea.com/index.php'

In [16]:
params = {'mid': 'best', 'page': 1}

In [19]:
result = requests.get(BASE_URL, params = params)

In [20]:
BeautifulSoup(result.text, 'lxml')

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?